In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
df = pd.read_csv('hh_ru_dataset.csv')

В целом смотрим только на следующие столбцы: final_state, work_experience_months, education_level, expected_salary, compensation_from, compensation_to в срезе profession

In [2]:
# отбрасываем дубликаты
df = df[['profession', 'final_state', 'expected_salary', 'compensation_from', 'compensation_to'
         , 'work_experience_months', 'education_level']]
df = df[(~df.profession.isna())&(~df.expected_salary.isna())&(~df.education_level.isna())&(~df.final_state.isna())&(~df.compensation_from.isna()) & (~df.compensation_to.isna()) & (~df.work_experience_months.isna())]


In [3]:
# отбрасывание статусов response-response, т.к. не знаем что с ними делать
df = df[df.final_state!='response']

In [4]:
# мб убрать зп неадекватную из ожидаемой и потолочной
df = df[df.work_experience_months <= 56*12] # отсекаем месяца работы выбросы

In [5]:
# total:
len(df)

162648

Новые столбцы

In [6]:
df['final_state'] = df['final_state'].map(lambda x: int(x=='invitation'))

In [7]:
edu_dict = {
    'secondary' : 0,
    'special_secondary': 1,
    'unfinished_higher':1,
    'bachelor':3,
    'higher':3,
    'master':4,
    'candidate':4,
    'doctor':4
}
df['education_level'] = df['education_level'].map(lambda x: edu_dict[x]+1)

In [8]:
management = ['Менеджер по продажам, менеджер по работе с клиентами', 'Бухгалтер', 'Менеджер по маркетингу, интернет-маркетолог',
             'Менеджер по персоналу', 'Менеджер по логистике, менеджер по ВЭД', 'Специалист по подбору персонала',
             'Менеджер по закупкам', 'Директор магазина, директор сети магазинов', 'Руководитель отдела продаж',
             'Офис-менеджер', 'Секретарь, помощник руководителя, ассистент', 'Руководитель проектов',
             'Финансовый менеджер', 'Администратор', 'Продавец-консультант, продавец-кассир', 
             'Менеджер по туризму', 'Координатор отдела продаж', 'Менеджер ресторана', 'Специалист по кадрам',
             'Менеджер/руководитель АХО']
design = ['Дизайнер, художник']
it = ['Программист, разработчик', 'Оператор ПК, оператор базы данных', 'Системный администратор',
     'SMM-менеджер, контент-менеджер', 'Тестировщик', 'Аналитик']
service = ['Официант, бармен, бариста', 'Специалист технической поддержки', 'Водитель', 'Охранник',
          'Курьер', 'Супервайзер', 'Кассир-операционист', 'Диспетчер', 'Торговый представитель', 
          'Упаковщик, комплектовщик', 'Повар, пекарь, кондитер', 'Кладовщик', 
          'Продавец-консультант, продавец-кассир', 'Начальник склада', 'Фитнес-тренер, инструктор тренажерного зала',
          'Товаровед', 'Оператор call-центра, специалист контактного центра', 'Уборщица, уборщик',
          'Разнорабочий', 'Мерчандайзер']
law = ['Делопроизводитель, архивариус', 'Юрисконсульт', 'Экономист', 'Юрист', 'Кредитный специалист']
engineering = ['Инженер-энергетик, инженер-электрик', 'Сварщик', 'Инженер-конструктор, инженер-проектировщик',
              'Слесарь, сантехник', 'Начальник производства', 'Начальник смены, мастер участка', 
              'Оператор производственной линии', 'Технолог', 'Машинист', 'Механик', 'Сервисный инженер, инженер-механик',
              'Инженер по охране труда и технике безопасности, инженер-эколог']
teacher = ['Учитель, преподаватель, педагог', 'Воспитатель, няня']
med = ['Психолог', 'Врач']
def f(x):
    if x in management:
        return "Менеджмент"
    if x in design:
        return "Дизайн"
    if x in it:
        return "IT"
    if x in service:
        return "Сфера услуг"
    if x in law:
        return "Право/Экономика"
    if x in engineering:
        return "Инженерия/Промышленность"
    if x in teacher:
        return "Образование"
    return "Медицина"
df['occupation'] = df.profession.map(f)

In [9]:
d = dict()
for i in df.profession.unique():
    temp = df[df.profession==i]
    d[i] = ((temp.compensation_to + temp.compensation_from)/2).median() #медиана средних зп
def temp(x):
    t = d[x['profession']]
    return (x['expected_salary'] - t) / t
df['delta_salary'] = df.apply(temp, 1)

In [10]:
# делаем модель
import statsmodels.api as sm
from statsmodels.genmod.generalized_linear_model import GLM
from statsmodels.genmod import families
import statsmodels.stats.tests.test_influence
import numpy as np

In [12]:
def show_cat(cat):
    temp = df[df.profession.map(lambda x: x in cat)]
    x = np.array(temp[['delta_salary', 'education_level', 'work_experience_months']])
    y = np.array(temp['final_state'])
    model_1 = sm.GLM(
        temp['final_state'],
        temp[['delta_salary', 'education_level', 'work_experience_months']],
        family=families.Binomial(),
    ).fit()
    print(model_1.summary())

In [19]:
show_cat(management)

                 Generalized Linear Model Regression Results                  
Dep. Variable:            final_state   No. Observations:                50980
Model:                            GLM   Df Residuals:                    50977
Model Family:                Binomial   Df Model:                            2
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                    nan
Date:                Sat, 16 Dec 2023   Deviance:                       62971.
Time:                        00:39:31   Pearson chi2:                 1.03e+05
No. Iterations:                     6   Pseudo R-squ. (CS):                nan
Covariance Type:            nonrobust                                         
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
delta_salary              -0

/usr/lib/python3/dist-packages/statsmodels/genmod/families/links.py:198: RuntimeWarning: overflow encountered in exp
  t = np.exp(-z)
/usr/lib/python3/dist-packages/statsmodels/genmod/families/family.py:1056: RuntimeWarning: divide by zero encountered in log
  special.gammaln(n - y + 1) + y * np.log(mu / (1 - mu + 1e-20)) +
/usr/lib/python3/dist-packages/statsmodels/genmod/families/family.py:1056: RuntimeWarning: invalid value encountered in multiply
  special.gammaln(n - y + 1) + y * np.log(mu / (1 - mu + 1e-20)) +


In [15]:
df.head(6)

,profession,final_state,expected_salary,compensation_from,compensation_to,work_experience_months,education_level,occupation,delta_salary
1,"Менеджер по продажам, менеджер по работе с кли...",1,80000.0,60000.0,110000.0,115,2,Менеджмент,-0.058824
2,"Менеджер по продажам, менеджер по работе с кли...",1,60000.0,45000.0,50000.0,52,2,Менеджмент,-0.294118
4,"Менеджер по продажам, менеджер по работе с кли...",1,70000.0,0.0,275000.0,42,2,Менеджмент,-0.176471
5,"Менеджер по продажам, менеджер по работе с кли...",1,25000.0,25000.0,50000.0,0,4,Менеджмент,-0.705882
11,"Менеджер по продажам, менеджер по работе с кли...",0,150000.0,181400.0,249800.0,73,1,Менеджмент,0.764706
16,"Менеджер по продажам, менеджер по работе с кли...",1,50000.0,80000.0,100000.0,28,4,Менеджмент,-0.411765
